In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import (
    LLMChain,
    LLMMathChain,
    TransformChain,
    SequentialChain
)

import re
import inspect

## Setting the Model

In [ ]:
with open("openai_api.txt", "r") as f:
    OPENAI_API = f.read()

llm = OpenAI(
    model_name = "gpt-3.5-turbo-instruct",
    openai_api_key = OPENAI_API
)

## Chains

Chains are the core of LangChain. They are simply a chain of components, executed in a `particular order`.

The simplest of these chains is the `LLMChain` that works by taking a user's input, passing it to the first element of the Chain, i.e., a PromptTemplate and then this prompt into an LLM.

By definition: _A chain is made up of links, which can be either primitives or other chains. Premitives can be either prompts, llms, utils or other chains_.

Chains are divided in three types: `Utility`, `Generic` and `Combine Documents` chains.

* `Utility` chains -- usually used to extract a specific answer from a llm with a very narrow purpose and are ready to be used out of the box.

* `Generic` chains -- used as building blocks for other chains but cannot be used out of the box on their own.

### `Utility Chains`


In [ ]:
llm_math = LLMMathChain(llm=llm, verbose=True)

llm_math("What is 13 raised to the .3432 power?")

In [ ]:
print(llm_math.prompt.template)

In [ ]:
## Printing the chain's `__call__()` method:

print(inspect.getsource(llm_math._call))

* Either returns an answer or it returns a Python code which we compile for an exacr answer to harder problems

* We get our first example of `chain composition`. We are using the *LLMMathChain* which in turn initializes and uses an *LLMChain* (a `Generic Chain`) when called.

* `Utility chains` usually follow this same basic structure: there is a prompt for constraining the llm to return a very specific type of response from a given query.

### `Generic Chains`

We will build a custom transform function to clean the spacing of our texts. We will then use this function to build a chain where we input our text and we expect a clean text as output.

In [ ]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]

    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}

In [ ]:
## Creating the Text Transformation Chain

clean_extra_spaces_chain = TransformChain(
    input_variables = ["text"],
    output_variables = ["output_text"],
    transform = transform_func
)

In [ ]:
clean_extra_spaces_chain.run('A random text  with   some irregular spacing.\n\n\n     Another one   here as well.')

In [ ]:
## Creating a Prompt Template to Pass it on Our Chain

template = """Paraphrase this text:
{output_text}

In the style of a {style}.

Paraphrase: """

prompt = PromptTemplate(
    template = template,
    input_variables = ["output_text", "style"]
)

In [ ]:
## Creating Base LLMChain object

style_paraphrase_chain = LLMChain(
    llm = llm,
    prompt = prompt,
    output_key = "final_output"
)

In [ ]:
## Passing the Output of TransformChain into LLMChain

sequential_chain = SequentialChain(
    chains = [clean_extra_spaces_chain, style_paraphrase_chain],
    input_variables = ["text", "style"],
    output_variables = ["final_output"]
)

In [ ]:
input_text = """
Chains allow us to combine multiple

components together to create a single, coherent application.

For example, we can create a chain that takes user input,       format it with a PromptTemplate,

and then passes the formatted response to an LLM. We can build more complex chains by combining     multiple chains together, or by


combining chains with other components.
"""

print(sequential_chain({'text': input_text, 'style': 'a 90s rapper'})["final_output"])